# 1. Install Dependencies and Setup

In [2]:
!pip install tensorflow opencv-python matplotlib

In [3]:
!pip list

Package                 Version
----------------------- -----------
absl-py                 2.1.0
asttokens               3.0.0
astunparse              1.6.3
certifi                 2024.8.30
charset-normalizer      3.4.0
colorama                0.4.6
comm                    0.2.2
contourpy               1.3.1
cycler                  0.12.1
debugpy                 1.8.9
decorator               5.1.1
executing               2.1.0
flatbuffers             24.3.25
fonttools               4.55.0
gast                    0.6.0
google-pasta            0.2.0
grpcio                  1.68.1
h5py                    3.12.1
idna                    3.10
ipykernel               6.29.5
ipython                 8.30.0
jedi                    0.19.2
jupyter_client          8.6.3
jupyter_core            5.7.2
keras                   3.7.0
kiwisolver              1.4.7
libclang                18.1.1
Markdown                3.7
markdown-it-py          3.0.0
MarkupSafe              3.0.2
matplotlib           

In [4]:
import tensorflow as tf
import os